## Get tweets!

In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient()

Pick one of the databases and create an object representing that database.

In [7]:
db = client["tweets"]


Create a new collection with a name of your choice.

In [10]:
db.create_collection("tweet_test")

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'tweets'), 'tweet_test')

Get one of the collections and create a new object representing it.

In [12]:
tweets_collection = db.get_collection('funky_test')

In [14]:
from pymongo import InsertOne, DeleteOne, ReplaceOne

Get some Twitter.  For access tokens, go to [Twitter's API site](https://apps.twitter.com) and load your consumer_key, consumer_secret, access_token, and access_token_secret into .twitter_config on your home directory.  See yesterday's Twittter notebook.

In [15]:
# pip install requests_oauthlib
import requests
from requests_oauthlib import OAuth1
import cnfg
from pprint import pprint
import json
import tweepy

In [16]:
config = cnfg.load(".twitter_config")

oauth = OAuth1(config["consumer_key"],
               config["consumer_secret"],
               config["access_token"],
               config["access_token_secret"])

In [17]:
# ?? this is an instance 

response = requests.get("https://api.twitter.com/1.1/statuses/user_timeline.json",
                        auth=oauth)
tweets = response.json()

In [20]:
privacy_search = 'facebook OR (data AND privacy) OR (data AND security) OR (data AND leak) OR (privacy AND violation)'

In [32]:
# Our collection: 
#tweets = db.new_tweet
#tweets_collection = db.get_collection('funky_test')
tweets_collection = db.get_collection('tweet_test')

In [34]:
tweets_collection.count()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


0

In [21]:
auth = tweepy.OAuthHandler(config["consumer_key"],
                           config["consumer_secret"])
auth.set_access_token(config["access_token"],
                      config["access_token_secret"])

api=tweepy.API(auth)

In [ ]:
max_tweets=2

#Tweepy Cursor handles pagination .. 

for tweet in tweepy.Cursor(api.search,q=privacy_search).items(max_tweets):
    pprint(tweet)

In [ ]:
#get tweets
#put them into mongo
#filter out things I want (id, text, user, location)

In [23]:
results=[]

for tweet in tweepy.Cursor(api.search,q=privacy_search).items(10):
    results.append(tweet)

In [29]:
results[0]._json['text']

'The Week in Tech: How Google and Facebook Spawned Surveillance Capitalism https://t.co/2FeQujNBvW'

In [24]:
for tweet in results:
    data={}
    data['tweet']=tweet.text.encode('utf-8') 
    data['datetime']=tweet.created_at

In [25]:
data

{'tweet': b'https://t.co/OsyCAX81A6',
 'datetime': datetime.datetime(2019, 2, 19, 23, 51, 27)}

In [38]:
for tweet in results:
    print (tweet._json['id'])
    tweets_collection.insert_one(tweet._json)

1098007186888888321
1098007186796507137
1098007185563410432
1098007185534140416
1098007184913371136
1098007184774975489
1098007184674377729
1098007184061939713
1098007184053542914
1098007184049426434


In [40]:
tweets_collection.find_one()

{'_id': ObjectId('5c6c99bfdfb4e30480cd1239'),
 'created_at': 'Tue Feb 19 23:51:27 +0000 2019',
 'id': 1098007186888888321,
 'id_str': '1098007186888888321',
 'text': 'The Week in Tech: How Google and Facebook Spawned Surveillance Capitalism https://t.co/2FeQujNBvW',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/2FeQujNBvW',
    'expanded_url': 'https://nyti.ms/2RYOEPK',
    'display_url': 'nyti.ms/2RYOEPK',
    'indices': [74, 97]}]},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 33994488,
  'id_str': '33994488',
  'name': 'Italian Politics',
  'screen_name': 'ItalianPolitics',
  'location': 'Rome',
  'description': 'News off the 

In [ ]:
for tweet in results:
    data={}
    data['tweet']=tweet.text.encode('utf-8') 
    data['datetime']=tweet.created_at
    tweets.insert_one(data)

In [ ]:
tweets.find_one()

In [ ]:
for tweet in tweets.find():
    print tweet['tweet']

In [ ]:
for tweet in tweets.find():
    print tweet['datetime']